# Gan

This notebook covers the process of building a Generative Adversarial Network (GAN) and it's application to the `MNIST` dataset.

In [40]:
import numpy as np
from pathlib import Path
from IPython.display import HTML

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision.transforms as T
import torchvision.utils as vutils
from torchvision.datasets import MNIST
from torch.nn.functional import binary_cross_entropy

import matplotlib.pyplot as plt
import matplotlib.animation as animation


if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")
print("using device", DEVICE)

using device cpu


As example `MNIST` dataset will be used.

In [2]:
TRAIN_DATASET = MNIST(
    Path("mnist_files"),
    download=True,
    train=True,
    transform=T.ToTensor()
)
DATA_LOADER = DataLoader(TRAIN_DATASET, batch_size=64)

## Generator

Following cell implements `Generator` class.

In [4]:
class Generator(nn.Module):
    '''
    A class that generates a picture from a set of random noise.

    Parameters
    ----------
    feature_map_size: int
        Feature map's size of the generator.
    number_channels: int
        Number of input channels.
    noize_size: int
        Size of the vector that is expected to be transformed to the picture by 
        the model.
    '''
    def __init__(
        self, 
        feature_map_size: int, 
        number_channels: int, 
        noize_size: int
    ) -> None:
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # (nz) x 1 x 1
            nn.ConvTranspose2d(
                in_channels=noize_size, 
                out_channels=feature_map_size * 2, 
                kernel_size=7,
                stride=1, 
                padding=0, 
                bias=False
            ),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.ReLU(True),

            # (feature_map_size*2) x 7 x 7
            nn.ConvTranspose2d(
                in_channels=feature_map_size * 2, 
                out_channels=feature_map_size, 
                kernel_size=4, 
                stride=2, 
                padding=1, 
                bias=False
            ),
            nn.BatchNorm2d(feature_map_size),
            nn.ReLU(True),

            # (feature_map_size) x 14 x 14
            nn.ConvTranspose2d(
                in_channels=feature_map_size, 
                out_channels=number_channels, 
                kernel_size=4, 
                stride=2, 
                padding=1, 
                bias=False
            ),
            nn.Tanh(),
            # (nc) x 28 x 28
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        '''
        Apply model to given data.

        Paramaters
        ----------
        input: torch.Tensor
            Tensor with size (n_samples, self.nz)

        Returns
        -------
        out: torch.Tensor
            Tensor that represents set of generated pictures.
        '''
        return self.main(input[:, :, None, None])

Now let's check what options there are to use it: to generator you have to pass vectors with random values. For each random vertor it will return picture.

In [5]:
generator = Generator(
    feature_map_size=64, 
    number_channels=1, 
    noize_size=100
)
generator(torch.randn(20, 100)).shape

torch.Size([20, 1, 28, 28])

## Discriminator

The discriminator is a model that tries to determine if a picture was created by the generator or not. The following cell defines the generator that we will use for this example.

In [6]:
class Discriminator(nn.Module):
    '''
    Realisation of the discriminator. Class that takes picture and generate scor
    which expresses how much the model thinks the picture is generated.

    Parameters
    ----------
    number_channels: int
        Number of channels in input.
    feature_map_size: int
        Feature map's size.
    '''

    def __init__(self, number_channels: int, feature_map_size: int):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # (number_channels) x 28 x 28
            nn.Conv2d(
                in_channels=number_channels, 
                out_channels=feature_map_size, 
                kernel_size=4, 
                stride=2, 
                padding=1, 
                bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),


            # (feature_map_size) x 14 x 14
            nn.Conv2d(
                in_channels=feature_map_size, 
                out_channels=feature_map_size * 2, 
                kernel_size=4, 
                stride=2, 
                padding=1, 
                bias=False
            ),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.LeakyReLU(0.2, inplace=True),


            # (feature_map_size*2) x 7 x 7
            nn.Conv2d(
                in_channels=feature_map_size * 2, 
                out_channels=1, 
                kernel_size=7, 
                stride=1, 
                padding=0, 
                bias=False
            ),
            
            nn.Sigmoid(),
            nn.Flatten(start_dim=0, end_dim=-1)
        )

    def forward(self, input):
        return self.main(input)

Consider how the discriminator works by passing a sample picture from the training data to it:

In [7]:
discriminator = Discriminator(
    number_channels=1, 
    feature_map_size=64
)
discriminator(TRAIN_DATASET[0][0][None, :, :, :])

tensor([0.4717], grad_fn=<ViewBackward0>)

We got a score that represents the model's prediction of whether the picture we passed was generated.

## Model fitting

We need to train the discriminator to determine if its input was original or generated by the generator. The following cell implements the optimization step with two core elements:

- Gradient accumulation to increase predicted scores for real images.
- Gradient accumulation to decrease predicted scores for generated images.

In [8]:
def discriminator_step(
    pictures: torch.Tensor,
    generation: torch.Tensor,
    discriminator: torch.nn.Module,
    optimizer: torch.optim.Optimizer
) -> tuple[float, float, float]:
    '''
    Step of the discriminator. Maximize log(D(x)) + log(1 - D(G(z))) - tries to 
    to improve the prediction that the real images have 1. scores and the 
    generated images have have 0. scores.

    Parameters
    ----------
    pictures: torch.Tensor
        Batch of real images that we're trying to imitate.
    generation: torch.Tensor
        Batch of generated images.
    discriminator: torch.nn.Module
        Model that we optimise.
    optimizer: torch.optim.Optimizer
        Optimizet that uses weights of the dicriminator.

    Return
    ------
    out: tuple[float, float, float]
        - Mean prediction for real images.
        - Mean predictoin for fake images.
        - Total loss value on both real and fake images.
    '''

    batch_size = pictures.shape[0]
    discriminator.zero_grad()

    # Gradient accumulation on real images
    # Model should predict scores close to 1
    label = torch.full((batch_size,), 1., dtype=torch.float, device=DEVICE)
    output = discriminator(pictures)
    errD_real = binary_cross_entropy(output, label)
    errD_real.backward()
    D_x = output.mean().item()

    # Gradient accumulation on fake images
    # Model should predicst scores close to 0
    label.fill_(0.)
    # Note: generation here bypasses the discriminator without gradient 
    # accumulation because we don't need generator gradients to optimize the 
    # discriminator.
    output = discriminator(generation.detach())
    errD_fake = binary_cross_entropy(output, label)
    errD_fake.backward()
    D_G_z1 = output.mean().item()


    errD = errD_real + errD_fake

    # Step based on accumulated gradients
    optimizer.step()

    return D_x, D_G_z1, errD

In this generator step, by calculating the loss (the difference between the discriminator’s judgment and the target “real” label), the generator learns the extent of improvement needed. This loss is used to compute gradients, guiding adjustments to the generator's weights to make future outputs more realistic.

In [9]:
def generator_step(
    generation: torch.Tensor,
    discriminator: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    generator: torch.nn.Module
) -> tuple[float, float]:
    '''
    Step of the generator.

    Parameters
    ----------
    genration: torch.Tensor
        Set of objects generated by generator.
    disciminator: torch.nn.Module
        Model which decision will determine behavior of the the generator.
    optimizer: torch.optim.Optimizer
        The optimizer that changes weights of the generator.
    generator: torch.nn.Module
        The model whose weights we are adjusting in this step.

    Returns
    -------
    out: tuple[float, float]
        - Mean prediction of the discriminator under generated data.
        - Loss value of the discriminator on generated picture.
    '''

    batch_size = generation.shape[0]
    generator.zero_grad()
    
    label = torch.full(
        size=(batch_size,), 
        fill_value=1.,
        dtype=torch.float,
        device=DEVICE
    )
    output = discriminator(generation)
    errG = binary_cross_entropy(output, label)
    errG.backward()


    D_G_z2 = output.mean().item()
    optimizer.step()

    return D_G_z2, errG

There are reasons to initialize model weights with specific random values. The following function implements normal distribution-based weight initialization for our models.

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

There are reasons to initialize model weights with specific random values. The following function implements normal distribution-based weight initialization for our models.

In [51]:
feature_map_size = 64
noise_size = 100

netG = Generator(
    feature_map_size=feature_map_size, 
    number_channels=1, 
    noize_size=noise_size
).to(DEVICE)
netG = netG.apply(weights_init)

netD = Discriminator(
    feature_map_size=feature_map_size, 
    number_channels=1
).to(DEVICE)
netD = netD.apply(weights_init)

lr = 0.001
beta1 = 0.5

# We'll draw images from the same input to compare results.
fixed_noise = torch.randn(64, noise_size, device=DEVICE)

optimizerD = torch.optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

img_list = []
G_losses = []
D_losses = []

num_epochs = 2

for epoch in range(num_epochs):
    for i, (pictures, _) in enumerate(DATA_LOADER, 0):
        pictures = pictures.to(DEVICE)
        batch_size = pictures.size(0)
        
        # Getting generated ("fake") picture that tries to trick discriminator
        noise = torch.randn(batch_size, noise_size, device=DEVICE)
        generation = netG(noise)

        D_x, D_G_z1, errD = discriminator_step(
            pictures=pictures,
            generation=generation,
            discriminator=netD,
            optimizer=optimizerD
        )

        # Maximizing for generator log(D(G(z)))
        D_G_z2, errG = generator_step(
            generation=generation,
            discriminator=netD,
            optimizer=optimizerG,
            generator=netG
        )

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if i % 50 == 0:
            print(
                f"[{epoch}/{num_epochs}][{i}/{len(DATA_LOADER)}]\t"
                f"Discriminator's loss: {errD.item():.4f}\t"
                f"Generator's loss: {errG.item():.4f}\t"
                f"D(x): {D_x:.4f}\t"
                f"D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}"
            )

        if (i % 500 == 0) or (
            (epoch == num_epochs - 1) and (i == len(DATA_LOADER) - 1)
        ):
            with torch.no_grad():
                generation = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(generation, padding=2, normalize=True))

[0/2][0/938]	Discriminator's loss: 1.1282	Generator's loss: 4.3460	D(x): 0.5763	D(G(z)): 0.3896 / 0.0169
[0/2][50/938]	Discriminator's loss: 0.1041	Generator's loss: 5.2774	D(x): 0.9319	D(G(z)): 0.0298 / 0.0071
[0/2][100/938]	Discriminator's loss: 0.3212	Generator's loss: 3.8745	D(x): 0.7893	D(G(z)): 0.0334 / 0.0337
[0/2][150/938]	Discriminator's loss: 0.0953	Generator's loss: 4.2726	D(x): 0.9408	D(G(z)): 0.0292 / 0.0201
[0/2][200/938]	Discriminator's loss: 0.1329	Generator's loss: 4.3186	D(x): 0.9537	D(G(z)): 0.0779 / 0.0235
[0/2][250/938]	Discriminator's loss: 0.1888	Generator's loss: 3.6425	D(x): 0.9805	D(G(z)): 0.1413 / 0.0429
[0/2][300/938]	Discriminator's loss: 0.2869	Generator's loss: 4.3238	D(x): 0.7740	D(G(z)): 0.0051 / 0.0238
[0/2][350/938]	Discriminator's loss: 0.5984	Generator's loss: 1.5117	D(x): 0.7295	D(G(z)): 0.2115 / 0.2792
[0/2][400/938]	Discriminator's loss: 0.6527	Generator's loss: 1.6094	D(x): 0.6866	D(G(z)): 0.1792 / 0.2659
[0/2][450/938]	Discriminator's loss: 0.2

Below is showen how generator evaluates each 500 steps.

In [52]:
fig = plt.figure(figsize=(8, 8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(
    fig=fig, 
    artists=ims, 
    interval=1000, 
    repeat_delay=1000, 
    blit=True
)
plt.close()

HTML(ani.to_jshtml())

Result pictures isn't briliant but at least they look like something handwritten.